In [10]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|')

/tmp/ipykernel_127493/3459715840.py:3: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|')


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Drop NA or fill
df = df.dropna(subset=['TotalClaims', 'TotalPremium'])

# Encode categoricals
for col in df.select_dtypes('object').columns:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

### Severity Model

In [7]:
# Filter only claimed
claims = df[df['TotalClaims'] > 0]
X = claims.drop(['TotalClaims'], axis=1)
y = claims['TotalClaims']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Build Models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

models = {
    'Linear': LinearRegression(),
    'RF': RandomForestRegressor(),
    'XGB': XGBRegressor()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    r2 = r2_score(y_test, preds)
    print(f"{name} - RMSE: {rmse}, R2: {r2}")

### SHAP Interpretation

In [33]:
pip install shap

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import shap
explainer = shap.Explainer(models['XGB'])
shap_values = explainer(X_test)
shap.plots.beeswarm(shap_values)